In [1]:
import pandas as pd

In [2]:
fff = pd.Series({
    'U22' : 97.4475,
    'V22' : 96.9400,
    'X22' : 96.4300,
    'Z22' : 96.2350,
    'F23' : 96.1150,
    'G23' : 96.0200,
    'H23' : 96.0050,
    'J23' : 95.9850,
    'K23' : 96.0050,
    'M23' : 95.5350,
    'N23' : 96.0650,
})

meetings = {
    'U22' : 21,
    'X22' : 2,
    'G23' : 14,
    'J23' : 22,
    'M23' : 21,
}

current_period = 'U22'
current_bucket = 225

In [3]:
# Create a new DataFrame indexed by FOMC meeting dates to store rate hike probabilities
hike_proba = pd.DataFrame(index=meetings.keys())

# Loop through each FOMC meeting date
for m in meetings:
    days = meetings[m] / 30  # Convert the meeting horizon from days to months (approximation)

    # Interpolate the implied rate at the meeting date using weighted average of adjacent monthly futures
    if days > 0.5:
        # If meeting is closer to the second month (e.g., end of the month), give more weight to the next month's contract
        fff1 = fff.shift(-1)[m]  # Price of the next futures contract
        fff0 = fff[m]            # Price of the current futures contract
        fff0 = (fff0 - (1 - days) * fff1) / days  # Solve for the implied short-term rate at the meeting
    else:
        # If meeting is closer to the first month (e.g., beginning of the month), give more weight to the current contract
        fff1 = fff[m]            # Price of the current futures contract
        fff0 = fff.shift(1)[m]   # Price of the previous futures contract
        fff1 = (fff1 - days * fff0) / (1 - days)  # Solve for implied rate

    # Convert futures prices to implied interest rates (since prices are quoted as 100 - rate)
    rate0 = 100 - fff0  # Implied rate at the start of the period
    rate1 = 100 - fff1  # Implied rate at the end of the period

    # Calculate the expected number of 25 bps hikes between the two rates
    hikes25bps = (rate1 - rate0) / 25 * 100  # Convert rate difference into multiples of 25 bps

    # Separate into integer and decimal part to distribute probability between two buckets
    breakdown = int(hikes25bps)  # Number of full 25bps hikes
    mod = hikes25bps % 1         # Fractional part to interpolate between two buckets

    # Distribute probability across the two nearest hike buckets
    # For example, if 1.3 hikes: 70% chance of +25bps, 30% chance of +50bps
    if breakdown == -4:
        hike_proba.at[m, -100] = mod
        hike_proba.at[m, -75] = 1 - mod
    elif breakdown == -3:
        hike_proba.at[m, -75] = mod
        hike_proba.at[m, -50] = 1 - mod
    elif breakdown == -2:
        hike_proba.at[m, -50] = mod
        hike_proba.at[m, -25] = 1 - mod
    elif breakdown == -1:
        hike_proba.at[m, -25] = mod        
        hike_proba.at[m, 0] = 1 - mod    
    elif breakdown == 0:
        hike_proba.at[m, 0] = 1 - mod
        hike_proba.at[m, 25] = mod
    elif breakdown == 1:
        hike_proba.at[m, 25] = 1 - mod
        hike_proba.at[m, 50] = mod
    elif breakdown == 2:
        hike_proba.at[m, 50] = 1 - mod
        hike_proba.at[m, 75] = mod
    elif breakdown == 3:
        hike_proba.at[m, 75] = 1 - mod
        hike_proba.at[m, 100] = mod

# Fill missing probabilities with 0 for clarity and consistency
hike_proba = hike_proba.fillna(0)


In [4]:
data = {}
# Initialize the cumulative probability dictionary with 0 bps move at 100%
proba = {0: 1.0}
# Loop through each FOMC meeting to build the probability tree step-by-step
for meeting in hike_proba.index:
    new_proba = {}
# Iterate over the current cumulative paths and their associated probabilities
    for previous_bps, previous_prob in proba.items():
# For each possible rate change, calculate the new cumulative bps and probability
        for delta_bps, prob in hike_proba.loc[meeting].items():
            if prob > 0:
# Calculate the new total rate hike from the path
                new_bps = previous_bps + delta_bps
# Update the cumulative probability for that new total
                new_proba[new_bps] = new_proba.get(new_bps, 0) + previous_prob * prob
# Set the new cumulative probabilities for the next meeting
    proba = new_proba
    data[meeting] = {hike + current_bucket : proba for hike, proba in proba.items()}

# Create a new DataFrame to store the cumulative probabilities
fomc_proba = pd.DataFrame.from_dict(data, orient="index")
fomc_proba = fomc_proba.loc[:, fomc_proba.notna().any()].astype(float)
# Replace missing values with 0
fomc_proba = fomc_proba.fillna(0).infer_objects(copy=False)
# Format the values as percentages for readability
fomc_proba = fomc_proba.map(lambda x: f"{x:.1%}")
fomc_proba = fomc_proba.reindex(meetings.keys())
# Sort the DataFrame columns for clean output
fomc_proba = fomc_proba.reindex(columns=sorted(fomc_proba.columns))

In [5]:
fomc_proba

,250,275,300,325,350,375,400,425
U22,0.0%,10.0%,90.0%,0.0%,0.0%,0.0%,0.0%,0.0%
X22,0.0%,0.0%,0.0%,8.1%,75.1%,16.7%,0.0%,0.0%
G23,0.0%,0.0%,0.0%,2.3%,27.4%,58.3%,11.9%,0.0%
J23,0.0%,0.0%,0.0%,0.3%,5.1%,30.8%,53.3%,10.6%
M23,0.2%,4.9%,30.0%,52.6%,11.8%,0.3%,0.0%,0.0%
